In [1]:
import data
import models
import cache
import stanity
import seaborn as sns
%matplotlib inline

/home/jacquelineburos/miniconda3/envs/immuneinf/lib/python3.5/site-packages/Cython/Distutils/old_build_ext.py:30: UserWarning: Cython.Distutils.old_build_ext does not properly handle dependencies and is deprecated.
  "Cython.Distutils.old_build_ext does not properly handle dependencies "


In [2]:
sns.set(context='talk')

In [3]:
import logging
cache_logger = logging.getLogger('cache')
cache_logger.setLevel(logging.INFO)

In [4]:
by = 'cell_type'
sample_n = 500

## load data

Load data as we did in previous notebooks, so we can compare predictive performance for particular observations

In [5]:
sample_df = cache.cached(models.prep_sample_df, sample_n=sample_n)

INFO:cache:prep_sample_df: cache_filename set to prep_sample_df.cached.sample_n_500.pkl
INFO:cache:prep_sample_df: Loading result from cache


In [6]:
stan_data = models.prep_stan_data(sample_df, by=by)

## model3 -- using poisson distribution

Load results from model3 from cache

In [7]:
model3 = models.get_model_file(model_name='model3')
fit3 = models.cached_stan_fit(file=model3, data=stan_data, model_name='model3', cache_only=True)

INFO:cache:Step 1: Get compiled model code, possibly from cache
INFO:cache:StanModel: cache_filename set to model3.cython_0_25_1.model_code_73307426795.pystan_2_12_0_0.stanmodel.pkl
INFO:cache:StanModel: Loading result from cache
INFO:cache:Step 2: Get posterior draws from model, possibly from cache
INFO:cache:sampling: cache_filename set to model3.cython_0_25_1.model_code_73307426795.pystan_2_12_0_0.stanfit.chains_4.data_40479835703.iter_2000.seed_1245502385.pkl
INFO:cache:sampling: Loading result from cache


In [8]:
models.print_stan_summary(fit3, pars='lp__')

              mean   se_mean         sd          2.5%           50%         97.5%      Rhat
lp__  7.840543e+07  0.864071  32.017284  7.840536e+07  7.840543e+07  7.840549e+07  1.001218


## model4 -- using negative binomial dist

Load results from fitting model4 from cache

In [9]:
model4 = models.get_model_file(model_name='model4')
fit4 = models.cached_stan_fit(file=model4, data=stan_data, model_name='model4', cache_only=True)

INFO:cache:Step 1: Get compiled model code, possibly from cache
INFO:cache:StanModel: cache_filename set to model4.cython_0_25_1.model_code_35447780597.pystan_2_12_0_0.stanmodel.pkl
INFO:cache:StanModel: Loading result from cache
INFO:cache:Step 2: Get posterior draws from model, possibly from cache
INFO:cache:sampling: cache_filename set to model4.cython_0_25_1.model_code_35447780597.pystan_2_12_0_0.stanfit.chains_4.data_40479835703.iter_2000.seed_1245502385.pkl
INFO:cache:sampling: Loading result from cache


In [10]:
models.print_stan_summary(fit4, pars='lp__')

              mean   se_mean         sd          2.5%           50%         97.5%      Rhat
lp__  7.976705e+07  0.911986  36.376698  7.976698e+07  7.976705e+07  7.976712e+07  0.999749


## Summarize LOO-psis for each model

Summarize Leave-One-Out (LOO) predictive performance for model3 & model4, using pareto-smoothed importance sampling (PSIS) method to approximate true CV performance.

In [11]:
loo3 = cache.cached(stanity.psisloo, log_likelihood=fit3.extract('log_lik')['log_lik'],
                   cache_filename='model3_psisloo.by_{}.sample_n_{}.pkl'.format(by, sample_n),
                   force=True)

INFO:cache:psisloo: cache_filename set to model3_psisloo.by_cell_type.sample_n_500.pkl
INFO:cache:psisloo: Loading result from cache


In [12]:
loo3.print_summary()

greater than 0.5    0.163746
greater than 1      0.088190
dtype: float64

The loo-summary for fit3 also suggests a poor model fit. We expect (hope) to have no more than ~ 5% of observations > 0.5, and even fewer greater than 1. These aren't hard endpoints; they are guidelines.

The interpretation is along the lines that:

1. There are too many observations exerting strong influence over our fit results, suggesting our model may be mis-parameterized by failing to account for these "extreme" values (extreme relative to model expectations)
2. This influence invalidates the approximation to LOO performance, which we will be runing next. Essentially, performance would be worse for these observations than estimated by this approximation.

In [13]:
loo4 = cache.cached(stanity.psisloo, log_likelihood=fit4.extract('log_lik')['log_lik'],
                   cache_filename='model4_psisloo.by_{}.sample_n_{}.pkl'.format(by, sample_n),
                   force=False)

INFO:cache:psisloo: cache_filename set to model4_psisloo.by_cell_type.sample_n_500.pkl
INFO:cache:psisloo: Loading result from cache


In [14]:
loo4.print_summary()

The history saving thread hit an unexpected error (OperationalError('database or disk is full',)).History will not be written to the database.


greater than 0.5    0.010730
greater than 1      0.001048
dtype: float64

These proportions are more in line with our expectations.

## Compare fit using model3 & model4

In [15]:
stanity.loo_compare(loo3, loo4)

{'diff': 1061108.3575595515, 'se_diff': 47160.146922995482}

A larger value suggests that model4 is a better fit than model3. How much better can be evaluated intuitively by comparing the magnitude of the difference to the SE of the difference.

The short answer for this comparison is .. *much* better.